# Day 3 - An AI parents assistant

An AI Parent Assistant that integrates with school systems and automates 60–80% of parent inquiries.

For this exercise we're providing a school website url which contains all the school information, providing static school information to aid in queries.

Features:
- Uses a school's website url to gain knowledge about the school
- Has username/password authentication

In [23]:
# imports

import os
import sys
import json

base_path = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(base_path)

print(base_path)

from scraper import fetch_website_links, fetch_website_contents
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

/Users/johnmboga/Documents/Applications/Andela AI Bootcamp/llm_engineering/week2


In [24]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenRouter API Key not set")

OpenRouter API Key exists and begins sk-or-v1


In [25]:
# Initialize

openrouter_url = "https://openrouter.ai/api/v1"
openrouter = OpenAI(base_url=openrouter_url,api_key=openrouter_api_key)
MODEL = 'gpt-4.1-mini'

In [26]:
#school website url
school_url = "https://www.makinischool.ac.ke/"
school_name = "Makini Group of Schools"

In [27]:
link_system_prompt = """
You are an assistant that extracts meaningful internal links from a school website.
Focus on pages related to fees, policies, admission, calendar, curriculum, handbook, contact info.
Ignore irrelevant pages like blogs, galleries, news archives.

Respond in JSON format:

{
    "links": [
        {"category_hint": "fees", "url": "https://example.com/fees"},
        {"category_hint": "admission", "url": "https://example.com/admission"}
    ]
}
"""

# USER prompt
def get_link_user_prompt(school_url):
    links = fetch_website_links(school_url) 

    link_user_prompt = f"""
    You are analyzing the following links from the school website: {school_url}

    Select the internal pages most useful for parents (fees, admission, calendar, policies, handbook, curriculum, contact info).
    Ignore irrelevant pages like blogs, galleries, news, careers, or login pages.

    Return ONLY valid JSON in this format:

    {{
        "links": [
            {{"category_hint": "fees", "url": "FULL_HTTPS_URL"}},
            {{"category_hint": "admission", "url": "FULL_HTTPS_URL"}}
        ]
    }}

    Links:
    """

    link_user_prompt += "\n".join(links)
    return link_user_prompt


In [ ]:
# method to select relevant links from the website
def select_relevant_links(school_url):
    messages = [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_link_user_prompt(school_url)}
        ]
    response = openrouter.chat.completions.create(
        model=MODEL,
        messages=messages,
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [36]:
def summarize_page(page_text, title="Page"):
    system_prompt = f"""
    You are a school content summarizer assistant.
    Summarize the following content concisely, keeping only important policies, fees, calendar, events, and contact info.
    Output plain text. Do not hallucinate or add anything not in the content.
    """
    user_prompt = f"Title: {title}\n\nContent:\n{page_text}"
    
    response = openrouter.chat.completions.create(
        model=MODEL,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        response_format={"type": "text"}
    )
    
    return response.choices[0].message.content


In [39]:
# method to fetch the insurance company's products 
def fetch_page_and_all_relevant_links(school_url):
    contents = fetch_website_contents(school_url)
    landing_summary = summarize_page(contents, title="Landing Page")    
    relevant_links = select_relevant_links(school_url)
    summaries = f"## Landing Page:\n\n{landing_summary}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        page_content = fetch_website_contents(link["url"])
        page_summary = summarize_page(page_content, title=f"Link: {link['category_hint']}")
        summaries += f"## {link['category_hint']}:\n\n{page_summary}\n"
    return summaries

In [44]:
school_url = "https://www.brookhouse.ac.ke/"
school_info  = fetch_page_and_all_relevant_links(school_url)

{'links': [{'category_hint': 'fees', 'url': 'https://www.brookhouse.ac.ke/admissions/fees'}, {'category_hint': 'admission', 'url': 'https://www.brookhouse.ac.ke/admissions/admissions-process'}, {'category_hint': 'admission', 'url': 'https://www.brookhouse.ac.ke/admissions/faqs'}, {'category_hint': 'calendar', 'url': 'https://www.brookhouse.ac.ke/school-life/calendar'}, {'category_hint': 'contact info', 'url': 'https://www.brookhouse.ac.ke/contact-us/enquire-now'}]}
Found 5 relevant links


In [45]:
# Again, I'll be in scientist-mode and change this global during the lab

system_prompt = """
You are a professional School Information Assistant for a Kenyan school.

# Core Role
Help parents and guardians understand official school information clearly and accurately.

You must:
- Provide clear and helpful explanations.
- Use only the information provided from the school’s website or official content.
- Avoid guessing or inventing policies.

# Scope
Focus strictly on:
- School policies
- Academic calendar
- Fees and payment information
- Uniform requirements
- School hours
- Contact details
- Events and activities

# Information Rules
- Only answer using the provided school content.
- If information is not available, say:
  "I do not have that information available from the school's official information."
- Do not speculate or create unofficial policies.

# Guardrails
- Do NOT fabricate rules, deadlines, or fees.
- Do NOT provide legal or financial advice.
- Clearly distinguish between confirmed information and unavailable information.

# Response Structure (When Applicable)

## Your Question
Briefly restate the parent’s question.

## Official Information
Provide the relevant school information clearly and concisely.

## Additional Guidance
- If appropriate, suggest contacting the school office for confirmation or clarification.
- Use ONLY the provided summarized school content below.
- Do not infer or add information not present in this text.
- If possible, mention which page the information came from (Landing Page, Admissions, Fees, etc.).


# Writing Style
- Professional and polite.
- Clear and simple English.
- Helpful and reassuring.
- Neutral and factual.

# Disclaimer
This response is based only on the school’s publicly available information and is for informational purposes only.
For official confirmation, please contact the school directly.
"""


In [46]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    dynamic_system_prompt = f"""
    {system_prompt}

    {school_info}
    """
    messages = [{"role": "system", "content": dynamic_system_prompt}] + history + [{"role": "user", "content": message}]
    stream = openrouter.chat.completions.create(model=MODEL, messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [48]:
def authenticate(username, password):
    users = {
        "ed@example.com": "bananas",
        "john@example.com": "pass123",
        "admin@example.com": "admin"
    }
    return users.get(username) == password

In [ ]:
gr.ChatInterface(fn=chat, type="messages", ).launch(inbrowser=True,auth=authenticate)